In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

получаем таблицу значений

In [2]:
df = pd.read_csv('winequality-white.csv', sep=';', decimal='.', na_values='NaN')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
df['quality'].value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

In [5]:
#  разделяю предикоторы и отклики
X = df.iloc[:, :-1] 
y = df.iloc[:, 11]  


In [6]:
#  разделяю на обучающую и тестовую выборку

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)  


In [14]:
from sklearn.tree import DecisionTreeClassifier

# Инициализируем модель
model = DecisionTreeClassifier(random_state=42,
                               # функция для impurity ('gini' или 'entropy')
                               criterion='gini',
                               # максимальная глубина дерева
                               max_depth=20,
                               # минимальное число элементов в узле для разбиения (может быть долей)
                               min_samples_split=3,
                               # минимальное число элементов в листе (может быть долей)
                               min_samples_leaf=3,
                               # минимальное значение дельты impurity
                               # min_impurity_decrease=0,
                               # веса для классов (можно дополнительно штрафовать за ошибку в нужных классах).
                               # поддерживает опцию 'balanced'.
                               class_weight=None,
                               # предварительная сортировка.
                               # ускоряет обучение на данных небольшого размера или с ограниченной глубиной дерева.
                               # иначе замедляет обучение.
                               presort=False
                              )

# Обучаем модель
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=20,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=3,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [15]:
pd.DataFrame({'feature': df.iloc[:, :-1].columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)


,feature,importance
10,alcohol,0.167789
1,volatile acidity,0.119127
5,free sulfur dioxide,0.102244
4,chlorides,0.083799
7,density,0.082270
8,pH,0.080358
2,citric acid,0.079948
6,total sulfur dioxide,0.077540
3,residual sugar,0.073586
9,sulphates,0.070719


In [16]:
# Строим предсказание модели
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)


In [17]:
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,3,4,5,6,7,8,9
3,0,1,7,4,2,0,0
4,1,14,42,20,5,3,0
5,2,52,405,226,37,3,0
6,15,33,245,621,147,27,0
7,1,1,57,183,186,17,0
8,0,0,8,35,33,13,0
9,0,0,0,2,1,0,0


In [18]:
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,3,4,5,6,7,8,9
3,5,0,0,0,1,0,0
4,1,56,8,8,4,1,0
5,2,19,643,51,12,5,0
6,4,13,64,991,29,9,0
7,2,4,32,43,350,4,0
8,1,3,8,16,14,44,0
9,0,1,0,0,0,1,0


Переобучения не происходит

In [29]:
print(metrics.classification_report(y_pred_test, y_test))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00        19
           4       0.16      0.14      0.15       101
           5       0.56      0.53      0.54       764
           6       0.57      0.57      0.57      1091
           7       0.42      0.45      0.43       411
           8       0.15      0.21      0.17        63
           9       0.00      0.00      0.00         0

   micro avg       0.51      0.51      0.51      2449
   macro avg       0.27      0.27      0.27      2449
weighted avg       0.51      0.51      0.51      2449

